In [ ]:
import requests
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
URL_16="https://web.archive.org/web/20160524083404/https://www.numbeo.com/cost-of-living/country_result.jsp?country=United+States"
# wayback machine referencing data from May of 2016
response=requests.get(URL_16)

In [ ]:
response.status_code

In [ ]:
response.text

In [ ]:
tomato_soup=BS(response.text)
print(tomato_soup.prettify())

In [ ]:
item = tomato_soup.findAll('td')[5]
item
#To quote Amanda Partlow, "why re-invent the wheel when you have something that already works"

In [ ]:
items = tomato_soup.find(class_= 'data_wide_table').findAll('td')
items

In [ ]:
#items.find(class_= 'tr_highlighted')
table_16=pd.read_html("https://web.archive.org/web/20160524083404/https://www.numbeo.com/cost-of-living/country_result.jsp?country=United+States")
table_16= table_16[2]
table_16.head()

### We got the first of the cost of living datasets from read_html, let's make it a dataframe

In [ ]:
cost16=pd.DataFrame(table_16)
cost16

## Set up Cost of Living datasets

###### We have the 2016 dataframe, set up the rest. 

##### The wayback machine is not consistent on WHEN it captures, but at least once a month. 
##### could not make a loop because of inconsistent days recorded for scraping 

In [ ]:
table_17=pd.read_html("https://web.archive.org/web/201704083404/https://www.numbeo.com/cost-of-living/country_result.jsp?country=United+States")
table_17= table_17[3]
table_18=pd.read_html("https://web.archive.org/web/20180429062837/https://www.numbeo.com/cost-of-living/country_result.jsp?country=United+States")
table_18= table_18[3]
table_19=pd.read_html("https://web.archive.org/web/20190403235143/https://www.numbeo.com/cost-of-living/country_result.jsp?country=United+States")
table_19= table_19[3]
table_20=pd.read_html("https://web.archive.org/web/20200508184633/https://www.numbeo.com/cost-of-living/country_result.jsp?country=United+States")
table_20= table_20[3]

##### Let's check to make sure all of that went through correctly and there are no problems.

In [ ]:
table_17.head()
#looks good

In [ ]:
table_17.tail()

In [ ]:
table_18.head()

In [ ]:
table_19.head()
# It WORKS! Finally starting to see some difference in prices too.

In [ ]:
table_20.head()

#### 2021 was not pulling correctly, so I singled that one out from the above column

In [ ]:
table_21=pd.read_html("https://web.archive.org/web/20210605054203/https://www.numbeo.com/cost-of-living/country_result.jsp?country=United+States")
table_21= table_21[2]
table_21.head()
#There was a slightly different index. Once located it produced the same table

In [ ]:
cost17=pd.DataFrame(table_17)
cost18=pd.DataFrame(table_18)
cost19=pd.DataFrame(table_19)
cost20=pd.DataFrame(table_20)
cost21=pd.DataFrame(table_21)

### That pulls in our Cost of living data.
###### The crime data was pulled in as CSV format from https://ucr.fbi.gov/crime-in-the-u.s/{year}

In [ ]:
cr16=pd.read_csv("../data/crime2016.csv", nrows=202)
header_row = cr16.iloc[2]
cr16 = pd.DataFrame(cr16.values[3:], columns=header_row)
cr16.head()

### Check the tail to make sure that we have exactly what we want. 

#### If we pulled in 2016 then we can pull in the others in the same way. 

In [ ]:
cr16.tail()

In [ ]:
cr17=pd.read_excel("../data/crime2017.xlsx", nrows=202)
header_row = cr17.iloc[2]
cr17 = pd.DataFrame(cr17.values[3:], columns=header_row)
cr17.head()

In [ ]:
cr18=pd.read_csv("../data/crime2018.csv", nrows=202)
header_row = cr18.iloc[2]
cr18 = pd.DataFrame(cr18.values[3:], columns=header_row)
cr18.head()

In [ ]:
cr15=pd.read_csv("../data/crime2015.csv", nrows=202, encoding = "ISO-8859-1")
header_row = cr15.iloc[2]
cr15 = pd.DataFrame(cr15.values[3:], columns=header_row)
cr15.head()

In [ ]:
cr20=pd.read_csv("../data/crime_2021.csv", nrows=202)
header_row = cr20.iloc[2]
cr20 = pd.DataFrame(cr20.values[3:], columns=header_row)
cr20.head()

#### Now all of the data is in place. Let's look at how they compare.

In [ ]:
cost=cost16.merge(cost17, how='inner', on='Restaurants')
cost=cost.rename(columns= {'[ Edit ] Avg._x':'2016_cost', '[ Edit ] Avg._y': '2017_cost', 'Range_x':'Range_2016', 'Range_y':'Range_2017'})
cost
# This looks pretty good. Let's put the rest together

In [ ]:
cost=cost.merge(cost18, how='inner')
cost=cost.rename(columns= {'[ Edit ]':'2018_cost', 'Range': 'Range_2018'})
cost.head(1)

In [ ]:
cost=cost.merge(cost19, how='inner')
cost=cost.rename(columns= {'[ Edit ]':'2019_cost', 'Range': 'Range_2019'})
cost.head(1)

In [ ]:
cost=cost.merge(cost20, how='inner')
cost=cost.rename(columns= {'[ Edit ]':'2020_cost', 'Range': 'Range_2020'})
cost=cost.merge(cost21, how='inner')
cost=cost.rename(columns= {'Edit':'2021_cost', 'Range': 'Range_2021'})
cost.head(1)

#### The cost table looks great and is complete. Let's do the same with the crime

In [ ]:
avg_costs=cost.drop(columns={'Range_2016','Range_2017','Range_2018','Range_2019','Range_2020','Range_2021'})
# I feel the most important rows for identifiers would be Meal, water, beer, and 1 bedroom apartment (both in and out of city)

In [ ]:
#narrow it down to tjust the columns we want
limited_cost=avg_costs[avg_costs['Restaurants'].isin(['Meal, Inexpensive Restaurant', 'Meal for 2 People, Mid-range Restaurant, Three...', 'Water (1.5 liter bottle)', 'Domestic Beer (0.5 liter bottle)', 'Apartment (1 bedroom) in City Centre', 'Apartment (1 bedroom) Outside of Centre'])]
limited_cost

In [ ]:
#find out what we are working with here
limited_cost.info()

In [ ]:
#first we have to drop the pesky symbols which will not convert
limited_cost=limited_cost.replace({'\s\$':''}, regex=True)
#the code also told me to remove commas, so let's do that while we're here.
limited_cost=limited_cost.replace({',':''}, regex=True)

In [ ]:
limited_cost

In [ ]:
# That's no good. We want floats or intergers, not objects. Let's convert them
convert = {'2016_cost': float,
           '2017_cost': float,
           '2018_cost': float,
           '2019_cost': float,
           '2020_cost': float,
           '2021_cost': float}

In [ ]:
limited_cost= limited_cost.astype(convert)
limited_cost.info()

In [ ]:
cost1=limited_cost

### Let's graph that to see how each price visually changed over time

In [ ]:
# Calculating Percentage
limited_cost['16-17'] = ((limited_cost['2017_cost']-limited_cost['2016_cost']) / limited_cost['2016_cost'])*100
limited_cost.head()

In [ ]:
limited_cost.info()

In [ ]:
limited_cost['17-18'] = ((limited_cost['2018_cost']-limited_cost['2017_cost']) / limited_cost['2017_cost'])*100
limited_cost['18-19'] = ((limited_cost['2019_cost']-limited_cost['2018_cost']) / limited_cost['2018_cost'])*100
limited_cost['19-20'] = ((limited_cost['2020_cost']-limited_cost['2019_cost']) / limited_cost['2019_cost'])*100
limited_cost['20-21'] = ((limited_cost['2021_cost']-limited_cost['2020_cost']) / limited_cost['2020_cost'])*100
limited_cost.head(1)

#### I feel good about that table with percent changes between each year.

In [ ]:
#Swap them around and drop the percentages
cost1=cost1[['Restaurants','2016_cost','2017_cost','2018_cost','2019_cost','2020_cost','2021_cost']]
cost1.set_index('Restaurants', inplace=True)
cost1=cost1.transpose()

In [ ]:
cost1.head()

In [ ]:
#Trying to normalize the data since apartment is throwing every other cost off
cheap=cost1[['Meal Inexpensive Restaurant','Water (1.5 liter bottle)','Domestic Beer (0.5 liter bottle)']]
apart=cost1[['Apartment (1 bedroom) in City Centre','Apartment (1 bedroom) Outside of Centre']]

In [ ]:
#I don't like this graph
cheap.plot.line(title="Cost of Goods");

plt.show;

In [ ]:
apart.plot.line(title="Cost of One Bedroom Housing");

plt.show;

## I have completed basic breakdown of goods over time. Let's look at the crime statistics. 

In [ ]:
violent6=cr16[['Area','Year','Population2','Violent crime3']]


In [ ]:
violent7=cr17[['Area','Year','Population2','Violent crime3']]


In [ ]:
violent8=cr18[['Area','Year','Population2','Violent crime3']]


In [ ]:
violent9=cr19[['Area','Year','Population2','Violent crime3']]


In [ ]:
violent0=cr20[['Area','Year','Population2','Violent crime3']]

In [ ]:
violent=pd.merge(violent6,violent7, how='left', on='Area')
violent

In [ ]:
cr16